This tutorial will show you how to use STP to look for an event, check its waveform availability, download and plot waveforms, and access phase picks. 

First, make sure you are running Jupyter from the same Python environment where PySTP is or will be installed. Download and install PySTP.

If you are using Google Colab, or you need to install PySTP and its dependencies in your local Python environment, uncomment and run the following line:

In [ ]:
# %pip install git+https://github.com/SCEDC/pystp.git

Once you have installed PySTP, import the ``pystp.STPClient`` module and any other needed modules. ``datetime`` is needed to search for events by date or timestamp.

In [ ]:
from pystp import STPClient
import datetime

# Allow plotting in the notebook.
%matplotlib inline

Connect to one of the STP servers by creating a new STPClient object and calling the `connect()` function. By default, it will connect to the first STP server, stp.gps.caltech.edu.

In [ ]:
stp = STPClient()
stp.connect()

# Finding Events

The `get_events` function searches for events and returns an [ObsPy catalog](https://docs.obspy.org/packages/autogen/obspy.core.event.Catalog.html), which is a container of events.

Let's look for a M4.2 event that occurred on January 25, 2023, near Malibu, CA, by searching for all M4-5 events that occurred between January 25 and January 26.

In [ ]:
events = stp.get_events(times=[datetime.datetime(2023, 1, 25), datetime.datetime(2023, 1, 26)], mags=[4, 5])
events

The events in the catalog can be accessed by index, as in a list. The network ID of the event is set in the field ``Event.resource_id``. The catalog ``events`` only contains one event, so let's store its ID in ``evid.``

In [ ]:
evid= events[0].resource_id
evid

# Listing Available Waveforms

The ``get_eavail`` function runs STP's ``eavail`` command to list available waveforms for an event. Long format, or 'l', will get full listings that include network code, station code, channel code, location code, waveform type, an archival status flag, start time, and duration of the waveform.

``get_eavail`` can return the result as a string or as a Python list, set by the ``as_list`` parameter.

Let's display all the waveforms available forevent 40161279.

In [ ]:
print(stp.get_eavail(evid, format='l', as_list=False))

Narrow down the search by looking at what's available for the station `CI.BAK`. `get_eavail` can search by network, station, or channel. Oracle wildcards ('\_' or '%') are accepted.

In [ ]:
stp.get_eavail(evid, format='l', net='CI', sta='BAK', chan="HH_")

Because ``as_list`` was not set, it defaulted to ``True``, and we got a Python list returned.

# Downloading Waveforms for an Event

The `get_trig` function downloads waveforms triggered by events and either returns the waveforms as a Python dictionary of ObsPy streams with event IDs as keys. Search parameters of `net` (network), `sta` (station), `chan` (SEED channel), `loc` (location code), and `radius` can be specified to only download data from certain stations or channels that meet the name criteria or are within a radius from the origin. "Blank blank" location codes should be passed in as "--". 

The next command downloads all waveforms from CI.BAKfor the above event and stores them in ``waveforms``.

In [ ]:
waveforms = stp.get_trig(evid, net='CI', sta='BAK', as_stream=False)
waveforms

In [ ]:
from obspy import read

In [ ]:
s = read('./40161279/40161279.CI.BAK.HNZ.sac')

In [ ]:
s
s.plot()

`waveforms` is a dictionary that maps the event ID to an ObsPy stream.

[ObsPy stream](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html) methods can be used to select traces from the stream and to do processing and plotting. The following line returns a new Stream that only contains data from CI.BAK.HH channels.

In [ ]:
waveforms[evid].select(network='CI', station='BAK', channel='HH*')

Let's use ObsPy's 'plot()' function to plot the waveforms from CI.BAK.HH.

In [ ]:
waveforms[evid].select(channel='HH*').plot()

# Downloading Phase Picks



In [ ]:
events[0].picks

In [ ]:
phase_catalog = stp.get_phases(evids=[evid])

In [ ]:
phase_catalog

In [ ]:
phase_catalog[0]

In [ ]:
phase_catalog[0].picks

# Disconnecting from the STP Server

When you are done, disconnect from the server. After disconnecting, you will need to run `connect()` again to use the STP server.

In [ ]:
stp.disconnect()